# MCP - Model Context Protocol
## Introduction
- Instead of coding the function calls to tools, we can also use an MCP server.
- This protocol helps the autodiscovery of tools and was initially conceived by Anthropic.
- Now it is way to plugin new tools into IDEs

More info via:
- https://github.com/modelcontextprotocol/servers
- https://modelcontextprotocol.io/docs/concepts/transports#server-sent-events-sse

In [30]:
%pip install -q fastmcp

Note: you may need to restart the kernel to use updated packages.


## A basic MCP security scanner

In [31]:
%pycat data/security_scanner_stdio.py

# math_server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Security Scanner")

@mcp.tool()
def scan(code : str) -> bool:
    """Scan code for security vulnerabilities"""

    if ("patrick" in code.lower()):
        return True
    else:
        return False
    
@mcp.prompt()
def scanner_prompt(code: str) -> str:
    """Prompt for scanning the code"""
    return "You are an amazing security scanner. Please scan the following code for security vulnerabilities: {code} \n\n" \

if __name__ == "__main__":
    mcp.run(transport="stdio")


## Execute the code and see what functions are available

In [32]:
"""MCP HTTP client example using MCP SDK."""

import asyncio
import sys
from typing import Any
from urllib.parse import urlparse

from mcp import StdioServerParameters
from mcp.client.session import ClientSession
from mcp.client.stdio import stdio_client 

from contextlib import AsyncExitStack


# https://github.com/slavashvets/mcp-http-client-example/blob/main/main.py

async def connect_to_stdio_server( server_script_path: str):
        """Connect to an MCP STDIO server
        
        Args:
            server_script_path: Path to the server script (.py or .js)
        """
        
        # Prepare execution
        server_params = StdioServerParameters(
            command="python",
            args=[server_script_path],
            env=None
        )
        
        # Initialize session
        exit_stack = AsyncExitStack()
        stdio_transport = await exit_stack.enter_async_context(stdio_client(server_params))
    
        stdio , write = stdio_transport
        session = await exit_stack.enter_async_context(ClientSession(stdio, write))
        
        await session.initialize()

         # List available information
        prompts = await session.list_prompts()
        resource_templates = await session.list_resource_templates()
        resources = await session.list_resources()
        response = await session.list_tools()
        tools = response.tools

        print("\nConnected to server with tools:", [tool.name for tool in tools])
        print("\nAvailable prompts:", [prompt for prompt in prompts])
        
        return session

async def main(mcp_code_file_path: str):
    """Connect to MCP server and list its capabilities.

    Args:
        mcp_code_file_path: Path to the MCP code file (.py)
    """

    try:
        await connect_to_stdio_server(mcp_code_file_path)

    except Exception as e:
        print(f"Error connecting to server: {e}")
        sys.exit(1)

mcp_code = "./data/security_scanner_stdio.py"
await main(mcp_code)



Connected to server with tools: ['scan']

Available prompts: [('meta', None), ('nextCursor', None), ('prompts', [Prompt(name='scanner_prompt', description='Prompt for scanning the code', arguments=[PromptArgument(name='code', description=None, required=True)])])]


## Start a linter corrector server using MCP

In [33]:
import subprocess

import os
os.environ['FASTMCP_PORT'] = '5432'
command = ['python', 'data/linter_corrector_sse.py']
# Start a background process using subprocess.Popen
process = subprocess.Popen(command, stdout=subprocess.PIPE,stderr=subprocess.PIPE)

## Connecting as an MCP client
Using the MCP SDK

In [34]:
%pip install -q mcp

Note: you may need to restart the kernel to use updated packages.


In [35]:
"""MCP HTTP client example using MCP SDK."""

import asyncio
import sys
from typing import Any
from urllib.parse import urlparse

from mcp.client.session import ClientSession
from mcp.client.sse import sse_client

# https://github.com/slavashvets/mcp-http-client-example/blob/main/main.py

def print_items(name: str, result: Any) -> None:
    """Print items with formatting.

    Args:
        name: Category name (tools/resources/prompts)
        result: Result object containing items list
    """
    print("", f"Available {name}:", sep="\n")
    items = getattr(result, name)
    if items:
        for item in items:
            print(" *", item)
    else:
        print("No items available")


async def main(server_url: str):
    """Connect to MCP server and list its capabilities.

    Args:
        server_url: Full URL to SSE endpoint (e.g. http://localhost:8000/sse)
    """
    if urlparse(server_url).scheme not in ("http", "https"):
        print("Error: Server URL must start with http:// or https://")
        sys.exit(1)

    try:
        async with sse_client(server_url) as streams:
            async with ClientSession(streams[0], streams[1]) as session:
                await session.initialize()
                print("Connected to MCP server at", server_url)
                print_items("tools", (await session.list_tools()))
                print_items("prompts", (await session.list_prompts()))
    except Exception as e:
        print(f"Error connecting to server: {e}")
        sys.exit(1)

In [36]:
# Now we can list the services from the network MCP service
url = "http://localhost:5432/sse"

await main(url)
#asyncio.run(main(url))

Connected to MCP server at http://localhost:5432/sse

Available tools:
 * name='correct' description='A linter good at correcting patrick code syntax' inputSchema={'properties': {'code': {'title': 'Code', 'type': 'string'}}, 'required': ['code'], 'title': 'correctArguments', 'type': 'object'}

Available prompts:
 * name='linter_prompt' description='Prompt good for linting code' arguments=[PromptArgument(name='code', description=None, required=True)]
